<h2><center> This is a Car Price Prediction Challenge Notebook</h2></center>
<figure>
<center><img src ="https://miro.medium.com/max/1296/1*kQBj7l-Y1WPZfX9nKIYL1Q.jpeg
" width = "750" height = '500' alt="Cryptojacking Detection Challenge"/>


*The objective of this challenge is*:
> Predicting the price of a car from various features, by using the datasets on Kaggle. Thereafter, it is crucial to submit the test prediction on kaggle platform.



## Table of contents:

1. [Importing Libraries](#Libraries)
2. [Loading Data](#Data)
3. [Exploratory Data analysis(EDA)](#Missing)
4. [Feature Engineering](#Engineering)
6. [Modelling](#Modelling)

## 1. Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor

## 2. Loading Data

In [ ]:
# Load train data
df_train = pd.read_csv("train.csv")
df_train.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,15000
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,7850


## 3. EDA (Exploratory Data Analysis)

In [ ]:
df_train.isnull().sum()

id              0
brand           0
model           0
model_year      0
milage          0
fuel_type       0
engine          0
transmission    0
ext_col         0
int_col         0
accident        0
clean_title     0
price           0
dtype: int64

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54273 entries, 0 to 54272
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            54273 non-null  int64 
 1   brand         54273 non-null  object
 2   model         54273 non-null  object
 3   model_year    54273 non-null  int64 
 4   milage        54273 non-null  int64 
 5   fuel_type     54273 non-null  object
 6   engine        54273 non-null  object
 7   transmission  54273 non-null  object
 8   ext_col       54273 non-null  object
 9   int_col       54273 non-null  object
 10  accident      54273 non-null  object
 11  clean_title   54273 non-null  object
 12  price         54273 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 5.4+ MB


In [ ]:
df_train.columns

Index(['id', 'brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'accident', 'clean_title',
       'price'],
      dtype='object')

In [ ]:
# Create a dictionary to hold unique values for each column
unique_counts = {col: df_train[col].nunique() for col in df_train.columns}
unique_counts


{'id': 54273,
 'brand': 53,
 'model': 1827,
 'model_year': 34,
 'milage': 3212,
 'fuel_type': 7,
 'engine': 1061,
 'transmission': 46,
 'ext_col': 260,
 'int_col': 124,
 'accident': 2,
 'clean_title': 1,
 'price': 1481}

In [ ]:
# Explore the engine column
df_train['engine'][0:5]

0        375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel
1    300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...
2         300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel
3    335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...
4        200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel
Name: engine, dtype: object

In [ ]:
# Extract the horsepower and engine size from 'engine' column
df_train['horsepower'] = df_train['engine'].str.extract(r'(\d+\.?\d*)HP')[0].astype(float)
df_train['engine_size'] = df_train['engine'].str.extract(r'(\d+\.?\d*)L')[0].astype(float)


In [ ]:
# drop the original 'engine' column
df_train.drop(columns=['engine'], inplace=True)

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54273 entries, 0 to 54272
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            54273 non-null  int64  
 1   brand         54273 non-null  object 
 2   model         54273 non-null  object 
 3   model_year    54273 non-null  int64  
 4   milage        54273 non-null  int64  
 5   fuel_type     54273 non-null  object 
 6   transmission  54273 non-null  object 
 7   ext_col       54273 non-null  object 
 8   int_col       54273 non-null  object 
 9   accident      54273 non-null  object 
 10  clean_title   54273 non-null  object 
 11  price         54273 non-null  int64  
 12  horsepower    50216 non-null  float64
 13  engine_size   53667 non-null  float64
dtypes: float64(2), int64(4), object(8)
memory usage: 5.8+ MB


In [ ]:
# Fill missing values in numerical columns with their mean values
numerical_columns = df_train.select_dtypes(include=['number']).columns
for col in numerical_columns:
    col_mean = df_train[col].mean()
    df_train[col].fillna(col_mean, inplace=True)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54273 entries, 0 to 54272
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            54273 non-null  int64  
 1   brand         54273 non-null  object 
 2   model         54273 non-null  object 
 3   model_year    54273 non-null  int64  
 4   milage        54273 non-null  int64  
 5   fuel_type     54273 non-null  object 
 6   transmission  54273 non-null  object 
 7   ext_col       54273 non-null  object 
 8   int_col       54273 non-null  object 
 9   accident      54273 non-null  object 
 10  clean_title   54273 non-null  object 
 11  price         54273 non-null  int64  
 12  horsepower    54273 non-null  float64
 13  engine_size   54273 non-null  float64
dtypes: float64(2), int64(4), object(8)
memory usage: 5.8+ MB


In [ ]:
df_train.head()

,id,brand,model,model_year,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,price,horsepower,engine_size
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,10-Speed A/T,Blue,Gray,None reported,Yes,11000,375.0,3.5
1,1,BMW,335 i,2007,80000,Gasoline,6-Speed M/T,Black,Black,None reported,Yes,8250,300.0,3.0
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,6-Speed A/T,Purple,Beige,None reported,Yes,15000,300.0,4.2
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500,335.0,3.0
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,A/T,White,Black,None reported,Yes,7850,200.0,3.8


## 4. Feature Enginneering Process

In [ ]:
# Extract the age of the model through 'model_year' column
df_train['model_age'] = 2024 - df_train['model_year']
df_train.drop(columns='model_year', inplace=True)

In [ ]:
df_train.head()

,id,brand,model,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,price,horsepower,engine_size,model_age
0,0,Ford,F-150 Lariat,74349,Gasoline,10-Speed A/T,Blue,Gray,None reported,Yes,11000,375.0,3.5,6
1,1,BMW,335 i,80000,Gasoline,6-Speed M/T,Black,Black,None reported,Yes,8250,300.0,3.0,17
2,2,Jaguar,XF Luxury,91491,Gasoline,6-Speed A/T,Purple,Beige,None reported,Yes,15000,300.0,4.2,15
3,3,BMW,X7 xDrive40i,2437,Hybrid,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500,335.0,3.0,2
4,4,Pontiac,Firebird Base,111000,Gasoline,A/T,White,Black,None reported,Yes,7850,200.0,3.8,23


In [ ]:
# Extract the categorical features
categorical_features = df_train.select_dtypes('object').columns
categorical_features = ['brand', 'model', 'fuel_type', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title']
# Extract the numerical features
numerical_features = df_train.select_dtypes(include=np.number).columns
numerical_features = ['milage', 'horsepower', 'engine_size', 'model_age']

In [ ]:
categorical_unique_counts = {col: df_train[col].nunique() for col in categorical_features}
categorical_unique_counts

{'brand': 53,
 'model': 1827,
 'fuel_type': 7,
 'transmission': 46,
 'ext_col': 260,
 'int_col': 124,
 'accident': 2,
 'clean_title': 1}

In [ ]:
# Instantiate Scaler and OnehotEncoder
one_hot = OneHotEncoder(handle_unknown='ignore', sparse=False)
scaler = StandardScaler()

In [ ]:
# Preprocess the data by applying the scaler and one_hot
numerical_features_scaled = scaler.fit_transform(df_train[numerical_features])
numerical_features_scaled

array([[ 0.03175858,  0.41662151, -0.16326211, -0.52032471],
       [ 0.14372824, -0.30498134, -0.53956775,  1.44787727],
       [ 0.37141245, -0.30498134,  0.36356578,  1.09002236],
       ...,
       [-0.62924091,  0.84958323, -0.08800098,  0.37431255],
       [-1.41132457,  0.        ,  0.21304353, -1.41496198],
       [-0.74790776, -0.68021482, -1.29217902, -1.05710707]])

In [ ]:
# Fit OneHotEncoder to learn unique values
one_hot.fit(df_train[categorical_features])

# Transform the categorical features
categorical_features_encoded = one_hot.transform(df_train[categorical_features])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
# Convert numerical_features_scaled and categorical_features_encoded to DataFrames
numerical_df = pd.DataFrame(numerical_features_scaled, columns=numerical_features)
categorical_df = pd.DataFrame(categorical_features_encoded, columns=one_hot.get_feature_names_out())

# Concatenate the two DataFrames along the columns (axis=1)
df_train_transformed = pd.concat([numerical_df, categorical_df], axis=1)
df_train_transformed.head()

,milage,horsepower,engine_size,model_age,brand_Acura,brand_Alfa,brand_Aston,brand_Audi,brand_BMW,brand_Bentley,...,int_col_Very Light Cashmere,int_col_Walnut,int_col_Whisper Beige,int_col_White,int_col_White / Brown,int_col_Yellow,int_col_–,accident_At least 1 accident or damage reported,accident_None reported,clean_title_Yes
0,0.031759,0.416622,-0.163262,-0.520325,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,0.143728,-0.304981,-0.539568,1.447877,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,0.371412,-0.304981,0.363566,1.090022,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,-1.393115,0.031767,-0.539568,-1.236035,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,0.757966,-1.267118,0.062521,2.521442,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [ ]:
df_train_transformed.describe()

,milage,horsepower,engine_size,model_age,brand_Acura,brand_Alfa,brand_Aston,brand_Audi,brand_BMW,brand_Bentley,...,int_col_Very Light Cashmere,int_col_Walnut,int_col_Whisper Beige,int_col_White,int_col_White / Brown,int_col_Yellow,int_col_–,accident_At least 1 accident or damage reported,accident_None reported,clean_title_Yes
count,5.427300e+04,5.427300e+04,5.427300e+04,5.427300e+04,54273.000000,54273.000000,54273.000000,54273.000000,54273.000000,54273.000000,...,54273.000000,54273.000000,54273.000000,54273.000000,54273.000000,54273.000000,54273.000000,54273.000000,54273.000000,54273.0
mean,2.631494e-17,-6.546006e-17,7.475539e-17,-9.714272e-17,0.010687,0.002653,0.000921,0.053839,0.135777,0.005233,...,0.000037,0.000074,0.000037,0.021595,0.000405,0.000129,0.019255,0.264902,0.735098,1.0
std,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,0.102824,0.051442,0.030339,0.225702,0.342554,0.072149,...,0.006070,0.008585,0.006070,0.145357,0.020130,0.011356,0.137420,0.441285,0.441285,0.0
min,-1.439421e+00,-2.460169e+00,-2.308204e+00,-1.593889e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,-8.020400e-01,-6.802148e-01,-5.395677e-01,-6.992522e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
50%,-1.315495e-01,-2.596157e-02,-1.632621e-01,-1.624698e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.0
75%,5.796392e-01,6.090489e-01,6.646103e-01,5.532400e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.0
max,6.583322e+00,6.622406e+00,3.524533e+00,7.352483e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0


In [ ]:
# split the data into train and test sets
X = df_train_transformed
y = df_train['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Selected features
features_selected = list(X.columns)

## 5. Modelling and Hyperparameter-tunning Process

> This modelling process instantiated different models by fitting to the training data and test them using test dataset. This facilitates in optimizing the model performance on the data towards the the lowest rmse.



In [ ]:
# Instantiate RandomForestRegressor
rf_regressor = RandomForestRegressor(random_state=42)

# Fit the model on training data
rf_regressor.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_regressor.predict(X_test)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("Root Mean Squared Error (RMSE):", rmse)


Root Mean Squared Error (RMSE): 53781.983020913416


In [ ]:
# Perfom feature selection
importances = rf_regressor.feature_importances_
feature_names = X_train.columns
feature_importances = pd.DataFrame({'feature': feature_names, 'importance': importances})

feature_importances = feature_importances.sort_values(by='importance', ascending=False)
feature_importances

,feature,importance
0,milage,0.235032
3,model_age,0.112216
1,horsepower,0.049789
2125,ext_col_Red,0.047448
1934,transmission_Transmission w/Dual Shift Mode,0.031027
...,...,...
1326,model_RX 350 F SPORT Appearance,0.000000
1329,model_RX 350 RX 350,0.000000
2132,ext_col_Rosso,0.000000
2136,ext_col_Ruby Red Metallic Tinted Clearcoat,0.000000


In [ ]:
# Top features I want to reuse
N = 50
top_features = feature_importances['feature'].head(N)

# Create a new dataset with only the top N features
X_train_selected = X_train[top_features]
X_test_selected = X_test[top_features]

# Train a new RandomForestRegressor with selected feature
rf_selected = RandomForestRegressor(n_estimators=100, random_state=42)
rf_selected.fit(X_train_selected, y_train)

# Predict and evaluate the model
y_pred = rf_selected.predict(X_test_selected)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error (RMSE):", rmse)

Root Mean Squared Error (RMSE): 52281.301643786624


In [ ]:
# Top features to reuse
N = 25
top_features = feature_importances['feature'].head(N)

# Create a new dataset with only the top N features
X_train_selected = X_train[top_features]
X_test_selected = X_test[top_features]

# Define the base models
base_models = [
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('gb', GradientBoostingRegressor(n_estimators=100, random_state=42)),
    ('xgb', XGBRegressor(n_estimators=100, random_state=42))
]

# Define the meta-model
meta_model = LinearRegression()

# Create the Stacking Regressor
stacking_model = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True
)

# Train the stacking model
stacking_model.fit(X_train_selected, y_train)

# Predict and evaluate the stacking model
y_pred = stacking_model.predict(X_test_selected)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error (RMSE):", rmse)


Root Mean Squared Error (RMSE): 49238.20656575377


In [ ]:
# Define parameter grid for grid search
param_grid = {
    'num_leaves': [31, 50, 100],
    'learning_rate': [0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'max_depth': [-1, 5, 10]
}

# Define LGBMRegressor
lgbm_model = LGBMRegressor(random_state=42)

# Perform grid search
grid_search = GridSearchCV(estimator=lgbm_model, param_grid=param_grid, cv=5, scoring='neg_root_mean_squared_error')
grid_search.fit(X_train_selected, y_train)

# Get the best model
best_lgbm_model = grid_search.best_estimator_

# Predict using the best model
y_pred_best = best_lgbm_model.predict(X_test_selected)

# Calculate RMSE
rmse_best = np.sqrt(mean_squared_error(y_test, y_pred_best))
print("RMSE with the best model:", rmse_best)

# Get feature importances
feature_importances = best_lgbm_model.feature_importances_

# Get indices of top N features
N = 10
top_feature_indices = feature_importances.argsort()[-N:][::-1]

# Get top features and their importance scores
top_features = X_train_selected.columns[top_feature_indices]
top_feature_importances = feature_importances[top_feature_indices]

# Print top features and their importance scores
print("\nTop Features:")
for feature, importance in zip(top_features, top_feature_importances):
    print(f"{feature}: {importance}")

#RMSE with the best model: 49034.4967305885

#op Features:
#milage: 834
#horsepower: 723
#engine_size: 495
#model_age: 487
#model_Sierra 1500 SLE Crew Cab: 143
#ext_col_Red: 100
#model_Sequoia Limited: 66
#accident_At least 1 accident or damage reported: 56
#transmission_4-Speed A/T: 55
#model_911 Carrera 4S: 52



Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

In [ ]:
X_train_best_selected = X_train[top_features]
X_test_best_selected = X_test[top_features]
X_train_best_selected

,milage,horsepower,engine_size,model_age,model_Sierra 1500 SLE Crew Cab,ext_col_Red,model_Sequoia Limited,accident_At least 1 accident or damage reported,transmission_4-Speed A/T,model_911 Carrera 4S
21234,1.788301,-0.593622,0.213044,-0.162470,0.0,0.0,0.0,0.0,0.0,0.0
49993,1.606368,-2.171527,-1.668485,0.553240,0.0,1.0,0.0,1.0,0.0,0.0
47016,-0.119998,-0.353088,-0.163262,0.016458,0.0,0.0,0.0,1.0,0.0,0.0
28922,0.247812,0.000000,-0.163262,-0.699252,0.0,0.0,0.0,0.0,0.0,0.0
22726,0.341870,-1.257497,-1.442701,0.553240,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
11284,-0.239477,-0.151039,-1.292179,-0.520325,0.0,0.0,0.0,0.0,0.0,0.0
44732,-0.877294,1.667400,2.470877,0.016458,0.0,0.0,0.0,0.0,0.0,0.0
38158,0.658896,-1.449925,-1.593224,0.553240,0.0,0.0,0.0,0.0,0.0,0.0
860,-1.173912,1.426866,-0.088001,0.911095,0.0,0.0,0.0,0.0,0.0,0.0


### Perform Ensembling Method on the best predictive models

In [ ]:
# Define the base models
base_models = [
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('gb', GradientBoostingRegressor(n_estimators=100, random_state=42)),
    ('xgb', XGBRegressor(n_estimators=100, random_state=42)),
    ('lgbm', LGBMRegressor(random_state=42))
]

# Define the meta_model
meta_model = LinearRegression()

# Instantiate the Stacking Regressor
stacking_mdl = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True
)

# Define parameter grid for each base model
param_grid = {
    'rf__n_estimators': [50, 100],
    'rf__max_depth': [None, 10, 20],
    'gb__n_estimators': [50, 100],
    'gb__learning_rate': [0.01, 0.1, 0.2],
    'xgb__n_estimators': [50, 100],
    'xgb__learning_rate': [0.01, 0.1, 0.2],
    'xgb__max_depth': [3, 6, 9],
    'lgbm__n_estimators': [50, 100],
    'lgbm__learning_rate': [0.01, 0.1, 0.2],
    'lgbm__num_leaves': [31, 50, 100]
}

# Wrap stacking model in GridSearchCV
grid_search = GridSearchCV(estimator=stacking_mdl, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Fit the grid search on the training data
grid_search.fit(X_train_best_selected, y_train)

# Get the best model from grid search
best_model = grid_search.best_estimator_

# Evaluate the best model
y_predictions = best_model.predict(X_test_best_selected)
rmse = np.sqrt(mean_squared_error(y_test, y_predictions))
print("Root Mean Squared Error (RMSE):", rmse)

# Print the best parameters
print("Best parameters found by grid search:", grid_search.best_params_)


Fitting 3 folds for each of 11664 candidates, totalling 34992 fits


In [ ]:
X_train_selected.columns

Index(['milage', 'model_age', 'horsepower', 'ext_col_Red',
       'transmission_Transmission w/Dual Shift Mode', 'ext_col_Silver',
       'model_RDX PMC Edition', 'model_Sierra 1500 SLE Crew Cab',
       'int_col_Beige', 'ext_col_Gray', 'model_Silverado 1500 LTZ',
       'transmission_A/T', 'engine_size', 'model_SL-Class SL500 Roadster',
       'fuel_type_Gasoline', 'ext_col_Black', 'fuel_type_Diesel',
       'transmission_4-Speed A/T', 'int_col_Black', 'ext_col_White',
       'model_Veyron 16.4 Grand Sport', 'model_911 Carrera 4S',
       'model_Sequoia Limited',
       'accident_At least 1 accident or damage reported',
       'transmission_6-Speed M/T'],
      dtype='object')

## **Submitting Test data predictions**

In [ ]:
df_test = pd.read_csv('/content/test.csv')
df_test.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,54273,Mercedes-Benz,E-Class E 350,2014,73000,Gasoline,302.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,White,Beige,None reported,Yes
1,54274,Lexus,RX 350 Base,2015,128032,Gasoline,275.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,Silver,Black,None reported,Yes
2,54275,Mercedes-Benz,C-Class C 300,2015,51983,Gasoline,241.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Blue,White,None reported,Yes
3,54276,Land,Rover Range Rover 5.0L Supercharged Autobiogra...,2018,29500,Gasoline,518.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,White,At least 1 accident or damage reported,Yes
4,54277,BMW,X6 xDrive40i,2020,90000,Gasoline,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes


In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36183 entries, 0 to 36182
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            36183 non-null  int64 
 1   brand         36183 non-null  object
 2   model         36183 non-null  object
 3   model_year    36183 non-null  int64 
 4   milage        36183 non-null  int64 
 5   fuel_type     36183 non-null  object
 6   engine        36183 non-null  object
 7   transmission  36183 non-null  object
 8   ext_col       36183 non-null  object
 9   int_col       36183 non-null  object
 10  accident      36183 non-null  object
 11  clean_title   36183 non-null  object
dtypes: int64(3), object(9)
memory usage: 3.3+ MB


In [ ]:
# Extract the horsepower and engine size from 'engine' column in test data
df_test['horsepower'] = df_test['engine'].str.extract(r'(\d+\.?\d*)HP')[0].astype(float)
df_test['engine_size'] = df_test['engine'].str.extract(r'(\d+\.?\d*)L')[0].astype(float)


In [ ]:
df_test.shape

(36183, 14)

In [ ]:
# drop the original 'engine' column
df_test.drop(columns=['engine'], inplace=True)

In [ ]:
# Extract the age of the model through 'model_year' column in test set
df_test['model_age'] = 2024 - df_test['model_year']
df_test.drop(columns='model_year', inplace=True)

In [ ]:
df_test.isnull().sum()

id                 0
brand              0
model              0
milage             0
fuel_type          0
transmission       0
ext_col            0
int_col            0
accident           0
clean_title        0
horsepower      2606
engine_size      405
model_age          0
dtype: int64

In [ ]:
df_test.shape

(36183, 13)

In [ ]:
# Fill missing values in numerical columns with their mean values
numerical_columns = df_test.select_dtypes(include=['number']).columns
for col in numerical_columns:
    col_mean = df_test[col].mean()
    df_test[col].fillna(col_mean, inplace=True)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36183 entries, 0 to 36182
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            36183 non-null  int64  
 1   brand         36183 non-null  object 
 2   model         36183 non-null  object 
 3   milage        36183 non-null  int64  
 4   fuel_type     36183 non-null  object 
 5   transmission  36183 non-null  object 
 6   ext_col       36183 non-null  object 
 7   int_col       36183 non-null  object 
 8   accident      36183 non-null  object 
 9   clean_title   36183 non-null  object 
 10  horsepower    36183 non-null  float64
 11  engine_size   36183 non-null  float64
 12  model_age     36183 non-null  int64  
dtypes: float64(2), int64(3), object(8)
memory usage: 3.6+ MB


In [ ]:
df_test.isnull().sum()

id              0
brand           0
model           0
milage          0
fuel_type       0
transmission    0
ext_col         0
int_col         0
accident        0
clean_title     0
horsepower      0
engine_size     0
model_age       0
dtype: int64

In [ ]:
# Extract the categorical features
categorical_features = df_test.select_dtypes('object').columns
categorical_features = ['brand', 'model', 'fuel_type', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title']
# Extract the numerical features
numerical_features = df_test.select_dtypes(include=np.number).columns
numerical_features = ['milage', 'horsepower', 'engine_size', 'model_age']

In [ ]:
categorical_unique_counts = {col: df_test[col].nunique() for col in categorical_features}
categorical_unique_counts

{'brand': 53,
 'model': 1791,
 'fuel_type': 7,
 'transmission': 44,
 'ext_col': 230,
 'int_col': 111,
 'accident': 2,
 'clean_title': 1}

In [ ]:
# Preprocess the data by applying the scaler and one_hot
numerical_features_scaled = scaler.fit_transform(df_test[numerical_features])

In [ ]:
# Fit OneHotEncoder to learn unique values
one_hot.fit(df_test[categorical_features])

# Transform the categorical features
categorical_features_encoded = one_hot.transform(df_test[categorical_features])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
# Convert numerical_features_scaled and categorical_features_encoded to DataFrames
numerical_df_test = pd.DataFrame(numerical_features_scaled, columns=numerical_features)
categorical_df_test = pd.DataFrame(categorical_features_encoded, columns=one_hot.get_feature_names_out())

# Concatenate the two DataFrames along the columns (axis=1)
df_test_transformed = pd.concat([numerical_df_test, categorical_df_test], axis=1)
df_test_transformed.head()

,milage,horsepower,engine_size,model_age,brand_Acura,brand_Alfa,brand_Aston,brand_Audi,brand_BMW,brand_Bentley,...,int_col_Tupelo,int_col_Very Light Cashmere,int_col_Walnut,int_col_White,int_col_White / Brown,int_col_Yellow,int_col_–,accident_At least 1 accident or damage reported,accident_None reported,clean_title_Yes
0,0.010268,-0.284346,-0.170472,0.190357,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,1.095406,-0.540978,-0.170472,0.011442,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,-0.404152,-0.864145,-1.298574,0.011442,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
3,-0.847479,1.768711,0.957630,-0.525302,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
4,0.345479,0.029316,-0.546506,-0.883131,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [ ]:
df_test_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36183 entries, 0 to 36182
Columns: 2243 entries, milage to clean_title_Yes
dtypes: float64(2243)
memory usage: 619.2 MB


In [ ]:
def retrieve_columns(df1, df2):

    # Get the list of columns in df1
    columns_df1 = df1.columns

    # Create a copy of df2 to avoid modifying the original DataFrame
    df2_aligned = df2.copy()

    # Ensure all columns in df1 are present in df2
    for col in columns_df1:
        if col not in df2_aligned.columns:
            df2_aligned[col] = 0

    # Select and order columns to match df1
    df2_selected = df2_aligned[columns_df1]

    return df2_selected

test_data = retrieve_columns(X_train_best_selected, df_test_transformed)
test_data.head()

,milage,horsepower,engine_size,model_age,model_Sierra 1500 SLE Crew Cab,ext_col_Red,model_Sequoia Limited,accident_At least 1 accident or damage reported,transmission_4-Speed A/T,model_911 Carrera 4S
0,0.010268,-0.284346,-0.170472,0.190357,0.0,0.0,0.0,0.0,0.0,0.0
1,1.095406,-0.540978,-0.170472,0.011442,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.404152,-0.864145,-1.298574,0.011442,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.847479,1.768711,0.957630,-0.525302,0.0,0.0,0.0,1.0,0.0,0.0
4,0.345479,0.029316,-0.546506,-0.883131,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36183 entries, 0 to 36182
Data columns (total 10 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   milage                                           36183 non-null  float64
 1   horsepower                                       36183 non-null  float64
 2   engine_size                                      36183 non-null  float64
 3   model_age                                        36183 non-null  float64
 4   model_Sierra 1500 SLE Crew Cab                   36183 non-null  float64
 5   ext_col_Red                                      36183 non-null  float64
 6   model_Sequoia Limited                            36183 non-null  float64
 7   accident_At least 1 accident or damage reported  36183 non-null  float64
 8   transmission_4-Speed A/T                         36183 non-null  float64
 9   model_911 Carrera 4S        

In [ ]:
# The 'id' is needed for the submission file
price_id = df_test['id']
price_id

0        54273
1        54274
2        54275
3        54276
4        54277
         ...  
36178    90451
36179    90452
36180    90453
36181    90454
36182    90455
Name: id, Length: 36183, dtype: int64

In [ ]:
# Add the predicted prices to the test set
test_data['target'] = stacking_mdl.predict(test_data)

In [ ]:
# add the 'id' to the test set
test_data['id'] = price_id

# select only the 'id' and predicted 'target', creating a new dataset
load = test_data[['id', 'target']]
load.to_csv('predicted_price.csv', index=False)
load.head()

,id,target
0,54273,26255.608723
1,54274,18754.370831
2,54275,27487.978493
3,54276,62392.501496
4,54277,37110.657394


In [ ]:
load.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36183 entries, 0 to 36182
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      36183 non-null  int64  
 1   target  36183 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 565.5 KB
